## How many cosmic muons traversing the TPC?

In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

How to get the vertical cosmic muon flux, $I_v$, see AnalyticCosmicSpectrum.ipynb.

Then use $I(\theta) = I_v\cos^2\theta$, regardless the momentum, to get the most conservative number of cosmic rays intersecting the TPC.

The TPC dimension (x, y, z) = (50, 60, 60)cm, while z is the vertical coordinate downward for convenience - different from the coordinate system used in the other studies.  The 0 is at the center of the (x, y) plane but the top side of the TPC.  I.e. $x \in [-25, 25]$, $y \in [-30, 30]$, $z \in [0, 60]$ cm.

In [2]:
# momentum range in GeV/c
pmin = 3
pmax = 100000

In [3]:
# Analytical cosmic ray flux from the paper
def dI_fixedTheta(p, theta):
    return 18/(p*np.cos(theta)+145)* (1./np.power((p+2.7/np.cos(theta)), 2.7))* (p+5)/(p+5/np.cos(theta))

In [4]:
# 1-D integral
theta = 0
Iv = integrate.quad(dI_fixedTheta, pmin, pmax, args = (theta))
Iv

(0.0035574258942180173, 1.0291962483084904e-08)

In [2]:
# Normalization factor of the probability density
F = integrate.quad( lambda x: x**2, 0, 1)

In [3]:
# Cosine distribution, cos^2(theta)
def x2(x):
    return x**2/F[0]

In [4]:
# CDF of the x^2 distribution
def cdf(costh):
    return (1.-costh**3)/(3*F[0])

In [5]:
# Inverse CDF of x^2
def inv_cdf(c):
    return np.cbrt(1-3*c*F[0])

In [6]:
def findLineIntersectionPointWithPlane(p0, v, p1, n):
    t = np.dot((p1-p0), n)/np.dot(v, n)
    return p0 + v*t

In [7]:
# test
p0 = np.array([0, -32, -2])
v = np.array([0, 1, 1])
p1 = np.array([0, -30, 0])
n = np.array([0, -1, 0])
x = findLineIntersectionPointWithPlane(p0, v, p1, n)
x

array([  0., -30.,   0.])

In [8]:
# Unit: cm
xHalf = 25.
yHalf = 30.
zFull = 60.
# Define the planes corresponding to the 6 faces by a point and a normal vector
p1s = np.array([[0., 0., 0.], [0., -yHalf,0.], [0., yHalf, 0.], [ -xHalf, 0., 0.], [xHalf, 0., 0.], [0., 0., zFull]])
norms = np.array([[0., 0., -1,], [0., -1., 0.], [0., 1., 0.], [-1., 0., 0.], [1., 0., 0.], [0., 0., 1.]])
# Define the borders of the 6 faces
eps = 1e-6
faces = np.array([[[-xHalf, xHalf], [-yHalf, yHalf], [-eps, eps]], 
                  [[-xHalf, xHalf], [-yHalf-eps, -yHalf+eps], [0., zFull]],
                  [[-xHalf, xHalf], [yHalf-eps, yHalf+eps], [0., zFull]],
                  [[-xHalf-eps, -xHalf+eps], [-yHalf, yHalf], [0., zFull]],
                  [[xHalf-eps, xHalf+eps], [-yHalf, yHalf], [0., zFull]],
                  [[-xHalf, xHalf], [-yHalf, yHalf], [zFull-eps, zFull+eps]]])

In [9]:
def findTraversing(x, face):
    
    if x[0] >= face[0][0] and x[0] <= face[0][1] and \
       x[1] >= face[1][0] and x[1] <= face[1][1] and \
       x[2] >= face[2][0] and x[2] <= face[2][1]:
        return True
    
    return False

In [10]:
# Test
findTraversing(np.array([-24, -30, 2]), faces[1])

True

In [11]:
def genSolidAngle(nSamples):
    uniformCosth = np.random.default_rng().uniform(0, 1, nSamples)
    SolidAngle = np.array([
        [costh, phi]
        for costh, phi in zip(inv_cdf(uniformCosth), np.random.default_rng().uniform(0, 2*np.pi, nSamples))])
    return SolidAngle

In [12]:
def getSinFromCos(costh):
    return np.sqrt(1 - costh**2)

In [13]:
# Unit: cm
# Define where to sample the cosmic muon flux
zPosition = -800
# zPosition = -10
nSolidAngles = 10000

In [14]:
def genXYPosition(xyBound, zPosition, nSteps):
    iStep = np.linspace(0, xyBound, nSteps+1)
    XPosYMin = np.array([[x, -xyBound, zPosition] for x in iStep ])
    XNegYMin = np.array([[x, -xyBound, zPosition] for x in -iStep])
    XPosYMax = np.array([[x, xyBound, zPosition]  for x in iStep ])
    XNegYMax = np.array([[x, xyBound, zPosition]  for x in -iStep])
    XMinYPos = np.array([[-xyBound, y, zPosition] for y in iStep ])
    XMinYNeg = np.array([[-xyBound, y, zPosition] for y in -iStep])
    XMaxYPos = np.array([[xyBound, y, zPosition]  for y in iStep ])
    XMaxYNeg = np.array([[xyBound, y, zPosition]  for y in -iStep])
    XYPosition = np.concatenate(( XPosYMin, XNegYMin, XPosYMax, XNegYMin, XMinYPos, XMinYNeg, XMaxYPos, XMaxYNeg))
    unique_check = np.unique(XYPosition, axis = 1)
    if len(XYPosition) != len(unique_check):
        print(f'{len(unique_check)} unique positions out of {len(XYPosition)} positions!')
    # print(f'{nXYPositions} points of cosmic rays being checked')
    
    return XYPosition

In [ ]:
# xyBounds = np.array([10., 20., 30., 40., 50., 100., 500., 1000., 5000. ])
# xyBounds = np.array([10., 100., 500.])
# xyBounds = np.array([7500., 10000,])
xyBounds = np.array([6000., 6500., 7000.])
nBounds = len(xyBounds)
nTraverse = np.zeros(nBounds)
nCounter = np.zeros(nBounds)

for ixyBound in range(nBounds):
    
    xyBound = xyBounds[ixyBound]
    XYPosition = genXYPosition(xyBound, zPosition, round(xyBound))

    for p0 in XYPosition:
        
        SolidAngles = genSolidAngle(nSolidAngles)
    
        for costh, phi in SolidAngles:
    
            # print(f'Cosmic ray {nCounter}')
    
            sinth = getSinFromCos(costh)
            v = np.array([sinth*np.cos(phi), sinth*np.sin(phi), costh])
            # print(f'v = {v}')
    
            for p1, norm, face in zip(p1s, norms, faces):
                x = findLineIntersectionPointWithPlane(p0, v, p1, norm)
                # print(f'x = {x}')
                doTraverse = findTraversing(x, face)
                if (doTraverse):
                    # print(f'It traverses! v = ({v})')
                    nTraverse[ixyBound] += 1
                    break

            nCounter[ixyBound] += 1

    # print(f'At {p0}, {nTraverse[index]} cosmic rays traversing the TPC out of {nCounter[index]}')

In [16]:
nTraverse

array([172.,  74.])

In [17]:
nCounter

array([6.0008e+08, 8.0008e+08])

In [18]:
r = np.array([ float(t)/float(c) for t, c in zip(nTraverse, nCounter)])
r

array([2.86628450e-07, 9.24907509e-08])

In [21]:
834363/880000

0.9481397727272727

In [19]:
result = np.array([ [xyBound, nT, nC, r] for xyBound, nT, nC, r in zip(xyBounds, nTraverse, nCounter, r)])
result

array([[7.50000000e+03, 1.72000000e+02, 6.00080000e+08, 2.86628450e-07],
       [1.00000000e+04, 7.40000000e+01, 8.00080000e+08, 9.24907509e-08]])

In [20]:
with open('nCosmicsInTPC2.npy', 'wb') as f:
    np.save(f, result)